# report

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from contract_select import contract_select
from hedge_calc import hedge_calc
from greek_calc import greek_calc, pnl_daily_calc
from position_adj_cost import position_adj_cost

In [ ]:
main_original = contract_select(select_type='main', date= '20101222', spy_spx='spx', contract_type='C', minAsk=0.2, minDistance = 0.03, maxDTE=15, num=3)
hedge_original = contract_select(select_type='hedge', date= '20101222', spy_spx='spx', contract_type='P', minAsk=0.2, minDistance = 0.06, num=4)
main_original.update(hedge_original)
main_original

In [ ]:
d = hedge_calc(main_original)
d

In [ ]:
for i in range(20):
    d = hedge_calc(d)
    d = pnl_daily_calc(d)
    print(d['daily_rtn'])